Projekt Streaming 

In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)

N = 800

df = pd.DataFrame({
	"user_id": range(1, N+1),
	"age": np.random.randint(18, 70, size = N),
	"country": np.random.choice(
		["PL", "DE", "UK", "FR", "ES"], size = N, p=[0.35, 0.2, 0.15, 0.15, 0.15]
	),
	"subscription_type": np.random.choice(
		["basic", "standard", "premium"], size = N, p=[0.45, 0.35, 0.2]
	),
	"monthly_fee": np.random.choice([29, 39, 59], size = N),
	"months_active": np.random.randint(1, 48, size = N),
	"hours_watched_per_month": np.random.normal(45, 18, size = N).clip(5, 140),
	"churned": np.random.choice([0, 1], size = N, p=[0.78, 0.22])
})

df

,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned
0,1,56,PL,standard,59,19,55.105237,0
1,2,69,ES,standard,29,12,55.915704,0
2,3,46,UK,standard,59,37,20.049157,0
3,4,32,PL,standard,29,42,71.082544,1
4,5,60,DE,basic,39,17,28.788352,0
...,...,...,...,...,...,...,...,...
795,796,68,UK,basic,29,24,64.029474,0
796,797,22,FR,premium,39,25,27.247959,0
797,798,64,UK,basic,29,45,70.534020,0
798,799,42,DE,standard,29,46,60.170962,1


ZADANIA - ETAP 1 (EDA)

1. Opis Danych

In [3]:
df.head()

,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned
0,1,56,PL,standard,59,19,55.105237,0
1,2,69,ES,standard,29,12,55.915704,0
2,3,46,UK,standard,59,37,20.049157,0
3,4,32,PL,standard,29,42,71.082544,1
4,5,60,DE,basic,39,17,28.788352,0


In [ ]:
df.shape

# 800 - wierszy, 8 kolumn

(800, 8)

In [ ]:
df.info() 	# typy danych, brak NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  800 non-null    int64  
 1   age                      800 non-null    int32  
 2   country                  800 non-null    object 
 3   subscription_type        800 non-null    object 
 4   monthly_fee              800 non-null    int64  
 5   months_active            800 non-null    int32  
 6   hours_watched_per_month  800 non-null    float64
 7   churned                  800 non-null    int64  
dtypes: float64(1), int32(2), int64(3), object(2)
memory usage: 43.9+ KB


2. Rozkłady:
- subscription_type
- country
- churned w %

In [9]:
# rozkład dla rodzaju subskrypcji

df["subscription_type"].value_counts()

subscription_type
basic       370
standard    298
premium     132
Name: count, dtype: int64

In [10]:
# rozkład dla państw

df["country"].value_counts()

country
PL    255
DE    168
FR    140
ES    120
UK    117
Name: count, dtype: int64

In [32]:
# rozkład dla churned w %

churned_procenty = df["churned"].mean() * 100
print(f"Churn rate: {churned_procenty:.2f}%")

Churn rate: 21.38%


3. Podstawowe pytania biznesowe
- Jaki jest churn rate? 
- Który plan ma najwyższy churn?
- Który kraj ma najwyższy churn?


In [17]:
# Jaki jest churn rate?

churn_rate = df["churned"].mean()

print(f"Churn rate wynosi: {churn_rate:.2f} %")

Churn rate wynosi: 0.21 %


In [33]:
# Który plan ma najwyższy churn ?

# Który plan traci najwięcej użytkowników?

df.groupby("subscription_type")["churned"].mean().sort_values(ascending=False)

subscription_type
premium     0.234848
standard    0.231544
basic       0.191892
Name: churned, dtype: float64

In [31]:
# Który kraj ma najwyższy churn?

df.groupby("country")["churned"].mean().sort_values(ascending=False)

country
UK    0.299145
FR    0.228571
PL    0.211765
DE    0.184524
ES    0.158333
Name: churned, dtype: float64

ZADANIA - ETAP 2 (Feature Engineering)
